In [1]:
!pip install rasterio

In [2]:
pip install scikeras

     |████████████████████████████████| 8.5 MB 793 kB/s eta 0:00:01
     |████████████████████████████████| 306 kB 16.9 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 0.17.0
    Uninstalling joblib-0.17.0:
      Successfully uninstalled joblib-0.17.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import rasterio as rio
from numba import jit
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

/usr/local/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Hyperparameter optimization

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
from typing import Tuple

def get_mnist() -> Tuple[np.ndarray, np.ndarray]:
    (X_train, y_train), _ = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 784)
    X_train = X_train.astype("float32")
    X_train /= 255
    return X_train, y_train

In [ ]:
from scipy.stats import loguniform, uniform
params = {"lr": loguniform(1e-3, 1e-1), "momentum": uniform(0, 1)}
X, y = get_mnist()

In [ ]:
from dask.distributed import Client
client = Client()

from dask_ml.model_selection import HyperbandSearchCV
search = HyperbandSearchCV(model, params, max_iter=27)
search.fit(X, y)

## Model wrapper

In [ ]:
# Define the CNN-LSTM architecture

# Construct the input layer with no definite frame size
inp = layers.Input(shape=(None, *x_train.shape[2:]))

# Construct three ConvLSTM2D layers with batch norm,
# followed by a Conv3D layer so that the output is
# the same shape as the original raster
x = layers.ConvLSTM2D(filters=num_filters,
                      kernel_size=(5, 5),
                      padding="same",
                      return_sequences=True,
                      activation="relu",)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(filters=num_filters,
                      kernel_size=(3, 3),
                      padding="same",
                      return_sequences=True,
                      activation="relu",)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(filters=num_filters,
                      kernel_size=(1, 1),
                      padding="same",
                      return_sequences=True,
                      activation="relu",)(x)
x = layers.Conv3D(filters=3, 
                  kernel_size=(3, 3, 3), 
                  activation="softmax", 
                  padding="same")(x)

# Build and compile the model
model = keras.models.Model(inp, x)
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam())

In [ ]:
from tensorflow import keras
from tensorflow.keras import models

neural_classifier = models.Sequential(
    [
        keras.layers.Dense(13, activation="relu", input_shape=(X_train.shape[1],)),
        keras.layers.Dense(26, activation="relu"),
        keras.layers.Dense(3, activation="softmax")
    ]
)

neural_classifier.summary()

In [ ]:
from scikeras.wrappers import KerasClassifier

scikeras_classifier = KerasClassifier(model=neural_classifier,
                                      optimizer="adam",
                                      loss=keras.losses.categorical_crossentropy,
                                      batch_size=8,
                                      epochs=100,
                                      verbose=0,
                                      validation_split=0.1
                                      )

In [ ]:
scikeras_classifier.fit(X_train, Y_train);

In [ ]:
Y_preds = scikeras_classifier.predict(X_test)
Y_probs = scikeras_classifier.predict_proba(X_test)

Y_preds[:5], Y_probs[:5]

In [ ]:
print("Test  Accuracy : {:.2f}".format(scikeras_classifier.score(X_test, Y_test)))
print("Train Accuracy : {:.2f}".format(scikeras_classifier.score(X_train, Y_train)))